In [110]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.neighbors import NearestNeighbors
import numpy as np
import textblob




In [3]:
business_json_path = '/Users/amelialei/yelp_dataset/yelp_academic_dataset_business.json'
bus_df = pd.read_json(business_json_path, lines = True)


In [4]:
bus_df = bus_df[bus_df['is_open'] == 1]


In [5]:
drop_cols = ['is_open', 'review_count']
bus_df = bus_df.drop(drop_cols, axis =1)


In [42]:
bus_df = bus_df[bus_df['categories'].str.contains('Bubble Tea|Coffee and Tea', case = False, na = False)] 


In [77]:
def get_wifi_info(attributes):
    if isinstance(attributes, dict): 
        info = attributes.get('WiFi', 'unknown') 
        if info != 'unknown':
            info = info.replace("u'", "").replace("'", "").strip()
    else:
        info = 'unknown'
    return info
    
bus_df['wifi'] = bus_df['attributes'].apply(get_wifi_info)

In [84]:
bus_df = bus_df.drop(['attributes', 'categories'], axis = 1)


KeyError: "['attributes', 'categories'] not found in axis"

In [86]:
review_json_path = '/Users/amelialei/yelp_dataset/yelp_academic_dataset_review.json'

In [87]:
size = 1000000
review = pd.read_json(review_json_path, lines=True, dtype = {'review_id': str, 'user_id': str, 'business_id': str, 'stars': int, 'date': str, 'text': str, 'useful': int, 'funny': int, 'cool': int}, chunksize = size)

In [88]:
chunk_list = []
for review_chunk in review:
    review_chunk = review_chunk.drop(['review_id', 'useful', 'funny', 'cool'], axis=1)
    review_chunk = review_chunk.rename(columns={'stars': 'review_stars'})
    chunk_merged = pd.merge(bus_df, review_chunk, on='business_id', how='inner')
    chunk_list.append(chunk_merged)
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

In [89]:
df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,hours,wifi,user_id,review_stars,text,date
0,i9eMSNpOA8BfHGZrM-9sZg,Kung Fu Tea,1469 Brace Rd,Cherry Hill,NJ,08034,39.88926,-75.015852,4.5,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",free,79nOboO-4_yNuQQ21EAU1A,5,Extremely Nice and clean I go there everyday l...,2021-05-18 22:04:32
1,i9eMSNpOA8BfHGZrM-9sZg,Kung Fu Tea,1469 Brace Rd,Cherry Hill,NJ,08034,39.88926,-75.015852,4.5,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",free,CSBRNusUV3D8t192J-W1yA,3,This experience was a little underwhelming. I...,2021-06-08 02:05:36
2,i9eMSNpOA8BfHGZrM-9sZg,Kung Fu Tea,1469 Brace Rd,Cherry Hill,NJ,08034,39.88926,-75.015852,4.5,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",free,Xc9cAaAlshYfBKOZXg2THA,5,Amazing. Refreshing drinks. There's a shortage...,2021-06-10 19:11:25
3,i9eMSNpOA8BfHGZrM-9sZg,Kung Fu Tea,1469 Brace Rd,Cherry Hill,NJ,08034,39.88926,-75.015852,4.5,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",free,R8kA79smxR1cy22U5ZQ56w,4,At first I questioned why there was another KF...,2021-05-18 19:18:18
4,i9eMSNpOA8BfHGZrM-9sZg,Kung Fu Tea,1469 Brace Rd,Cherry Hill,NJ,08034,39.88926,-75.015852,4.5,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",free,bhAB5wo9iLsaExSw4apR4g,4,This is the second Kung Fu tea that has sprung...,2021-07-30 00:24:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8576,uTUTULYSAmHGK7w5DDw5iA,A Cup Of Tea,115 N 9th St,Philadelphia,PA,19107,39.95393,-75.154491,4.5,"{'Monday': '11:0-22:30', 'Tuesday': '11:0-22:3...",free,0mlZTI1wIWOck0jHwkVUGg,5,Super delicious! They have unique flavors here...,2021-08-24 21:46:59
8577,uTUTULYSAmHGK7w5DDw5iA,A Cup Of Tea,115 N 9th St,Philadelphia,PA,19107,39.95393,-75.154491,4.5,"{'Monday': '11:0-22:30', 'Tuesday': '11:0-22:3...",free,O_jHz6PqR0RbMClMYlE86A,4,Good quality of tea and some unique options. S...,2021-10-14 23:09:36
8578,uTUTULYSAmHGK7w5DDw5iA,A Cup Of Tea,115 N 9th St,Philadelphia,PA,19107,39.95393,-75.154491,4.5,"{'Monday': '11:0-22:30', 'Tuesday': '11:0-22:3...",free,Gzvxy0mNhlypxPri8iRBMQ,5,"AMAZING,\n\n\n\nthats all i can say\nIf you go...",2021-12-28 20:24:14
8579,uTUTULYSAmHGK7w5DDw5iA,A Cup Of Tea,115 N 9th St,Philadelphia,PA,19107,39.95393,-75.154491,4.5,"{'Monday': '11:0-22:30', 'Tuesday': '11:0-22:3...",free,ahSrNB_GPtkFwSpP93QF-g,3,"Good tea, interesting milk foam. I wish it wou...",2021-09-24 17:35:16


In [90]:
df['text'] = (df['text']
              .str.lower()
              .replace(r'\band\b|\bor\b|\bthe\b|\bis\b|\bto\b', '', regex=True)
              .replace(r'[^A-Za-z\s]', '', regex=True))

ps = PorterStemmer()
def tokenize_and_stem(review):
    tokens = word_tokenize(review)
    stemmed_tokens = [ps.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

cleaned_reviews = df['text'].apply(tokenize_and_stem)
df['text'] = cleaned_reviews
df['sentiment'] = df['text'].apply(lambda review: TextBlob(review).sentiment.polarity)

In [92]:
df['normalized_stars'] = (df['stars'] - df['stars'].min())/(df['stars'].max()-df['stars'].min())
df['normalized_review_stars'] = (df['review_stars'] - df['review_stars'].min())/(df['review_stars'].max()-df['review_stars'].min())
df = pd.get_dummies(df, columns=['wifi'], drop_first=True)

In [113]:
user_item_matrix = df.pivot_table(index='user_id', columns='business_id', values='review_stars')
user_item_matrix = user_item_matrix.apply(lambda col: col.fillna(col.mean()), axis=0)

In [115]:
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(user_item_matrix.values)

user_id = '79nOboO-4_yNuQQ21EAU1A'
distances, indices = knn_model.kneighbors([user_item_matrix.loc[user_id]], n_neighbors=6)

similar_users = indices.flatten()[1:]
similar_users_ids = user_item_matrix.index[similar_users]
cafe_recs = user_item_matrix.loc[similar_users_ids].mean(axis=0).sort_values(ascending=False).head()
print(cafe_recs)



business_id
YTxOgn-vquErIXTA97Lrlw    5.000000
hemQ0_nE8du-ednYNYMvLw    5.000000
xt-GigN6sRFQXKCIyIOCYA    4.909091
ToW6MFXu2C6N04FualtFxQ    4.900000
_OmjicQbWgNNKJUTYMixjg    4.900000
dtype: float64
